

# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.

In [ ]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GRUMemory(nn.Module):

    def __init__(self, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        # GRU: input size = N+1 (one-hot length), hidden size = hidden_size
        self.gru = nn.GRU(input_size=N+1, hidden_size=hidden_size, batch_first=True)
        # Decoder: map hidden state → N+1 logits
        self.decoder = nn.Linear(hidden_size, N+1)

    def forward(self, x):
        """
        x: (batch_size, seq_length, N+1) one-hot inputs
        returns: logits (batch_size, seq_length, N+1)
        """
        # GRU pass
        out, _ = self.gru(x)   # out: (batch_size, seq_length, hidden_size)
        # Decode each timestep
        logits = self.decoder(out)  # (batch_size, seq_length, N+1)
        return logits

    @torch.no_grad()
    def test_run(self, s):
        """
        s: input string (letters a-z)
        returns: output string with echo delay learned
        """
        # map chars → indices
        idxs = torch.tensor([ord(c) - ord('a') + 1 for c in s], dtype=torch.int64)
        onehot = idx_to_onehot(idxs.unsqueeze(0))  # shape (1, seq_len, N+1)

        # run through network
        logits = self.forward(onehot)
        preds = torch.argmax(F.log_softmax(logits, dim=-1), dim=-1)  # (1, seq_len)

        # convert back indices → chars
        result = []
        for i in preds[0]:
            if i.item() == 0:
                result.append(" ")   # 0 = blank
            else:
                result.append(chr(ord('a') + i.item() - 1))
        return "".join(result)


## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [ ]:
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-DELAY], result[DELAY:]):
      correct += int(c1 == c2)
    total += len(s) - DELAY

  return correct / total

In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

start_time = time.time()

# ----------------------------
# Model setup
# ----------------------------
HIDDEN_SIZE = 256
BATCH_SIZE = 128
EPOCHS = 8
LR = 1e-3
SEQ_LEN = 15
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

train_ds = EchoDataset(delay=DELAY, seq_length=SEQ_LEN, size=50000)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)


model = GRUMemory(hidden_size=HIDDEN_SIZE).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# ----------------------------
# Training loop
# ----------------------------
for epoch in range(EPOCHS):
    model.train()
    for seq, target in train_loader:
        x = idx_to_onehot(seq, k=N+1).float().to(DEVICE)
        y = target.to(DEVICE)

        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits.view(-1, N+1), y.view(-1))
        loss.backward()
        optimizer.step()

end_time = time.time()
duration = end_time - start_time

# ----------------------------
# Final test
# ----------------------------
accuracy = test_model(model)

assert duration < 600, f'execution took {duration:.2f} seconds, which is longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy:.3f}, need 0.99'
print(f'tests passed ✅ | duration: {duration:.2f}s | accuracy: {accuracy:.3f}')

tests passed ✅ | duration: 73.70s | accuracy: 1.000


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [ ]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

### Model

And the model.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VariableDelayGRUMemory(nn.Module):

    def __init__(self, hidden_size, max_delay):
        super().__init__()
        self.hidden_size = hidden_size
        self.max_delay = max_delay

        # GRU input size = alphabet one-hot size (N+1) + delay embedding
        self.delay_embed = nn.Embedding(max_delay + 1, 8)   # small delay embedding
        self.gru = nn.GRU(input_size=(N+1) + 8,
                          hidden_size=hidden_size,
                          batch_first=True)
        self.decoder = nn.Linear(hidden_size, N+1)

    def forward(self, x, delays):
        """
        x: (batch, seq_len, N+1)
        delays: (batch,) integer delays
        returns: logits (batch, seq_len, N+1)
        """
        # embed delays
        delay_emb = self.delay_embed(delays)  # (batch, 8)

        # repeat delay embedding across time steps
        delay_expanded = delay_emb.unsqueeze(1).expand(-1, x.size(1), -1)  # (batch, seq_len, 8)

        # concat with input
        x_cat = torch.cat([x, delay_expanded], dim=-1)  # (batch, seq_len, N+1+8)

        out, _ = self.gru(x_cat)  # (batch, seq_len, hidden_size)
        logits = self.decoder(out)  # (batch, seq_len, N+1)
        return logits

    @torch.no_grad()
    def test_run(self, s, delay):
        """
        s: input string (letters a-z)
        delay: int, chosen delay
        returns: output string
        """
        # map chars to indices
        idxs = torch.tensor([ord(c) - ord('a') + 1 for c in s], dtype=torch.int64).unsqueeze(0)
        onehot = idx_to_onehot(idxs)  # (1, seq_len, N+1)

        # delay as tensor
        d = torch.tensor([delay], dtype=torch.int64)

        # forward pass
        logits = self.forward(onehot.float(), d)
        preds = torch.argmax(F.log_softmax(logits, dim=-1), dim=-1)[0]

        # convert indices → chars
        result = []
        for i in preds:
            if i.item() == 0:
                result.append(" ")
            else:
                result.append(chr(ord('a') + i.item() - 1))
        return "".join(result)


### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [ ]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time

start_time = time.time()

# ----------------------------
# Hyperparameters
# ----------------------------
MAX_DELAY = 8
SEQ_LENGTH = 20
HIDDEN_SIZE = 256
BATCH_SIZE = 128
EPOCHS = 10
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ----------------------------
# Dataset & Loader
# ----------------------------
train_ds = VariableDelayEchoDataset(max_delay=MAX_DELAY, seq_length=SEQ_LENGTH, size=100000)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, drop_last=True)

# ----------------------------
# Model, Loss, Optimizer
# ----------------------------
model = VariableDelayGRUMemory(hidden_size=HIDDEN_SIZE, max_delay=MAX_DELAY).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# ----------------------------
# Training Loop
# ----------------------------
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for seq, delays, target in train_loader:
        # one-hot encode sequence
        x = idx_to_onehot(seq, k=N+1).float().to(DEVICE)      # (batch, seq_len, N+1)
        y = target.to(DEVICE)                                 # (batch, seq_len)
        d = delays.to(DEVICE)                                 # (batch,)

        optimizer.zero_grad()
        logits = model(x, d)                                  # (batch, seq_len, N+1)
        loss = criterion(logits.view(-1, N+1), y.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    acc = test_variable_delay_model(model, seq_length=SEQ_LENGTH)
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {avg_loss:.4f} | Test Acc: {acc*100:.2f}%")

# ----------------------------
# Final check
# ----------------------------
end_time = time.time()
duration = end_time - start_time
final_acc = test_variable_delay_model(model, seq_length=SEQ_LENGTH)

assert duration < 600, f'executing took {duration:.2f}s, which is longer than 10 mins'
assert final_acc > 0.99, f'accuracy is too low, got {final_acc:.3f}'
print(f"✅ tests passed | duration {duration:.2f}s | accuracy {final_acc:.3f}")


Epoch 1/8 | Loss: 1.4997 | Test Acc: 62.30%
Epoch 2/8 | Loss: 0.5572 | Test Acc: 81.64%
Epoch 3/8 | Loss: 0.2897 | Test Acc: 89.37%
Epoch 4/8 | Loss: 0.1746 | Test Acc: 94.38%
Epoch 5/8 | Loss: 0.1057 | Test Acc: 96.72%
Epoch 6/8 | Loss: 0.0671 | Test Acc: 97.73%
Epoch 7/8 | Loss: 0.0427 | Test Acc: 98.64%
Epoch 8/8 | Loss: 0.0294 | Test Acc: 99.17%


AssertionError: executing took 824.64s, which is longer than 10 mins